## Check power consumption
Using package `pynvml` power is measured for Maelstrom Deliverable 3.7 on the Juelich Super Computer.    
Here, quick consistency checks are performed and values aggregated for the report. 

In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [ ]:
!ls ../../scripts/finetune_deberta/mlflow_projects/deberta_rain_classifier

In [ ]:
# some_file.py
import sys

# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, "../../scripts/finetune_deberta/mlflow_projects/deberta_rain_classifier")
HARDWARE = "amd"
import utils_scripts

if HARDWARE == "amd":
    import utils_energy_amd as utils_energy
else:
    import utils_energy_nvidia as utils_energy

In [ ]:
ENERGY_FOLDERS_BASE = "/home/kehlert/code/a2/cluster/benchmarks3.7/ap2_run/"
JUBE_IDS = [
    # 111,
    # 112,
    # 113,
    70,
    71,
    76,
]
NGPUS = 1
CLUSTER = "E4"

In [ ]:
!source ~/.bashrc
for jid in JUBE_IDS:
    # !mkdir ${CLUSTER}/${jid}
    os.makedirs(f"{CLUSTER}/{jid}/", exist_ok=True)
    !rsync -Pvra kehlert@172.18.19.216:$ENERGY_FOLDERS_BASE/$(printf "%06d" $jid)/000000_submit/work/trained_models/dataset_split_thresh0M2/power/ $CLUSTER/$jid/

In [ ]:
n_gpus = 1
for jid in JUBE_IDS:
    files = glob.glob(f"{CLUSTER}/{jid}/*.csv")
    if len(files) != 1:
        print(f"Not only single csv found rather:\n{files=}")
    power = utils_scripts.load_power(files[0])
    # print(power)
    print(f"\nLooking at Jube ID {jid}:")
    print(f"Total energy [Wh]: {utils_scripts.total_power_consumption_Wh(power, n_gpus=n_gpus):.2f}")
    print(f"Max power [W]: {utils_scripts.max_power_W(power, n_gpus=n_gpus):.2f}")
    print(f"Max aggregate power [W]: {utils_scripts.max_aggegrate_power_W(power, n_gpus=n_gpus):.2f}")
    print(f"Mean aggregate power [W]: {utils_scripts.mean_aggegrate_power_W(power, n_gpus=n_gpus):.2f}")
    # print(f"Total power consumption per device [Wh]: {utils_scripts.total_power_consumption_Wh_per_device(power)}")

In [ ]:
power

In [ ]:
power_gpus = utils_scripts.get_power_gpus(power, n_gpus)

for i in range(np.shape(power_gpus)[1]):
    plt.plot(power.index.values[:-1], power_gpus[:, i], label=i)
plt.legend()

In [ ]:
power_gpus

In [ ]:
plt.hist(np.diff(power.index.values)[:, None])

In [ ]:
def total_power_consumption_Wh_per_device(power):
    return np.sum(power[power.columns].values[1:] * np.diff(power.index.values)[:, None] / 3600, axis=0)


def average_consumption_W_per_device(power):
    return np.mean(power[power.columns].values[1:], axis=0)


def total_power_consumption_Wh(power):
    return sum(total_power_consumption_Wh_per_device(power))


total_power_consumption_Wh_per_device(power), total_power_consumption_Wh(power)

In [ ]:
"-".join(np.array(average_consumption_W_per_device(power), str))